In [ ]:
import pandas as pd
import numpy as np

import unidecode
import re


from collections import Counter

import spacy

from nltk.corpus import stopwords
import gensim.downloader as api


from sklearn.cluster import KMeans


def vectorize(list_of_docs, model):
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token, cnt in Counter(tokens).items():
            if token in model and len(token)>2 and cnt >2:
                try:
                    vectors.append(model[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec.tolist())
        else:
            features.append(zero_vector)
    return features

def project_data_preparation(text, wv):
    data = df[:100]['text']
    reg = re.compile('[^a-zA-Z ]')
    data=data.apply(lambda x: ' '.join(reg.sub(' ', unidecode.unidecode(str(x))).lower().strip().split()))
    my_stop = stopwords.words('english')+['article', 'page', 'papper', 'abstract'
                                      , 't', 's', 'by', 'awarded', 'conference', 'will', 'of', 'for', 'problem']
    sp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    data = data.apply(lambda x:  [w.lemma_ for w in sp(x) if (w.text not in my_stop)])
    vectorize(data, wv)
    return vectorize(data, wv)

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics


import gensim 

import itertools



from gensim.models import Word2Vec

In [ ]:
#nltk.download('stopwords')

In [ ]:
#spacy.cli.download("en_core_web_sm")

In [ ]:
df['d2v_sp'] = vectorize(df['sp'], wv)
df['sum'] = df['d2v_sp'].apply(lambda x: sum(x))
df_droped=df[df['sum'] != 0]

предобученный гугл

In [ ]:
cur_model = KMeans(n_clusters=24, random_state=42).fit(pd.DataFrame(df_droped['d2v_sp'].tolist()))
for i in range(len(cur_model.cluster_centers_)):
    print ('\nClaster_'+str(i)+f": {np.unique(cur_model.labels_, return_counts=True)[1][i]}")
    print(wv.similar_by_vector(cur_model.cluster_centers_[i], topn=10))

In [ ]:
df_droped['label'] = cur_model.labels_